In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
pd.set_option('float_format', '{:f}'.format)
# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df  = pd.read_csv('../input/test.csv')

In [3]:
print('Old size: %d' % len(train_df))
train_df = train_df[(train_df.kills <= 35) & (train_df.assists <= 13) & (train_df.boosts <= 16) & (train_df.damageDealt <= 3500) 
                   & (train_df.DBNOs <= 35) & (train_df.headshotKills <= 22) & (train_df.killStreaks <= 10) & (train_df.assists <= 13)
                   & (train_df.longestKill <= 1000) & (train_df.revives <= 20) & (train_df.rideDistance <= 28000) & (train_df.roadKills <= 15) 
                   & (train_df.swimDistance <= 3500) & (train_df.teamKills <= 10) & (train_df.vehicleDestroys <= 3) & (train_df.walkDistance <= 13000) 
                   & (train_df.weaponsAcquired <= 60)]

print('New size: %d' % len(train_df))

def headshot_precent(data):
    data['headPerc']=data['headshotKills']/data['kills']
    data['roadPerc']=data['roadKills']/data['kills']
    data['totalDistance']=data['rideDistance']+data['swimDistance']+data['walkDistance']
    data['rideDistancePerc']=data['rideDistance']/data['totalDistance']
    data['swimDistancePerc']=data['swimDistance']/data['totalDistance']
    data['walkDistancePerc']=data['walkDistance']/data['totalDistance']
    
    data.fillna(0,axis=1,inplace=True)                                          
    return data.head()
headshot_precent(train_df)
headshot_precent(test_df)

Old size: 4357336
New size: 4357226


,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,maxPlace,numGroups,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,headPerc,roadPerc,totalDistance,rideDistancePerc,swimDistancePerc,walkDistancePerc
0,47734,1659463,47734,0,0,100.000000,1,0,0,59,1000,0,0,0.000000,50,45,0,0.000000,0,0.000000,1,0,421.500000,7,1500,0.000000,0.000000,421.500000,0.000000,0.000000,1.000000
1,47735,1659508,47735,0,1,400.000000,2,0,3,9,1200,4,2,15.420000,48,47,1,0.000000,0,0.000000,0,0,655.800000,4,1526,0.000000,0.000000,655.800000,0.000000,0.000000,1.000000
2,47736,1659555,47736,0,0,0.000000,0,0,0,72,1216,0,0,0.000000,29,28,0,0.000000,0,0.000000,0,0,74.580000,1,1475,0.000000,0.000000,74.580000,0.000000,0.000000,1.000000
3,47737,1659621,47737,0,0,68.600000,0,0,0,68,1177,0,0,0.000000,97,92,0,0.000000,0,0.000000,0,0,167.200000,2,1464,0.000000,0.000000,167.200000,0.000000,0.000000,1.000000
4,47738,1659675,47738,0,1,370.500000,3,0,1,45,1013,1,1,5.968000,27,26,0,0.000000,0,0.000000,0,0,146.700000,3,1505,0.000000,0.000000,146.700000,0.000000,0.000000,1.000000


In [4]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV

In [5]:
dataset = pd.concat([train_df, test_df], sort=True)
df = dataset.groupby(['matchId','groupId']).agg('max')
drop_features = ["Id", "groupId", "matchId"]
feats = [f for f in train_df.columns if f not in drop_features]

dataset = dataset[drop_features].merge(df[feats], how='left', on=['matchId','groupId'])
dataset.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,maxPlace,numGroups,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc,headPerc,roadPerc,totalDistance,rideDistancePerc,swimDistancePerc,walkDistancePerc
0,0,24,0,0,5,284.200000,2,0,9,17,1050,2,2,65.320000,29,28,1,762.100000,0,0.000000,0,0,1857.000000,4,1458,0.857100,0.000000,0.000000,2619.100000,0.430443,0.000000,0.709022
1,1,440875,1,1,0,149.700000,2,1,1,82,1072,1,1,13.550000,26,23,1,0.000000,0,0.000000,0,0,197.300000,3,1525,0.040000,1.000000,0.000000,197.300000,0.000000,0.000000,1.000000
2,2,878242,2,1,2,143.800000,2,1,7,55,1470,2,1,121.700000,28,28,1,511.400000,0,0.000000,0,0,3248.000000,5,1583,0.740700,0.500000,0.000000,3248.000000,0.172165,0.000000,1.000000
3,3,1319841,3,0,0,95.880000,0,0,0,86,1069,0,0,0.000000,97,94,0,0.000000,0,0.000000,0,0,21.490000,1,1489,0.114600,0.000000,0.000000,21.490000,0.000000,0.000000,1.000000
4,4,1757883,4,0,1,141.300000,1,0,1,59,1192,1,1,9.144000,47,41,1,0.000000,0,0.000000,1,0,1077.000000,4,1512,0.521700,0.000000,0.000000,1077.000000,0.000000,0.000000,1.000000


In [10]:
params = {
    'num_leaves': 144,
    'learning_rate': 0.1,
    'n_estimators': 800,
    'max_depth':13,
    'max_bin':55,
    'bagging_fraction':0.8,
    'bagging_freq':5,
    'feature_fraction':0.9
    }

In [11]:
import gc
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['winPlacePerc'].notnull()]
    test_df = df[df['winPlacePerc'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()

    drop_features = ["winPlacePerc", "Id", "groupId", "matchId"]
    feats = [f for f in train_df.columns if f not in drop_features]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['winPlacePerc'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['winPlacePerc'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['winPlacePerc'].iloc[valid_idx]

        # LightGBM parameters
        reg = LGBMRegressor(num_leaves=params['num_leaves'], learning_rate=params['learning_rate'], 
                            n_estimators=params['n_estimators'], max_depth=params['max_depth'],
                            max_bin = params['max_bin'], bagging_fraction = params['bagging_fraction'], 
                            bagging_freq = params['bagging_freq'], feature_fraction = params['feature_fraction']
                           )

        reg.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)],
            eval_metric= 'mae', verbose= 50, early_stopping_rounds= 200)

        oof_preds[valid_idx] = reg.predict(valid_x)
        sub_preds += reg.predict(test_df[feats]) / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = reg.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d MAE : %.6f' % (n_fold + 1, mean_absolute_error(valid_y, oof_preds[valid_idx])))
        del reg, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full MAE score %.6f' % mean_absolute_error(train_df['winPlacePerc'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['winPlacePerc'] = sub_preds
        test_df[['Id', 'winPlacePerc']].to_csv("submission.csv", index= False)
    return feature_importance_df

In [12]:
feat_importances = kfold_lightgbm(dataset, num_folds=5, stratified=False, debug=False)


Starting LightGBM. Train shape: (4357226, 32), test shape: (1867913, 32)
Training until validation scores don't improve for 200 rounds.
[50]	training's l1: 0.0525139	valid_1's l1: 0.0525999
[100]	training's l1: 0.0508103	valid_1's l1: 0.0509779
[150]	training's l1: 0.0501044	valid_1's l1: 0.0503577
[200]	training's l1: 0.049676	valid_1's l1: 0.0499999
[250]	training's l1: 0.0493324	valid_1's l1: 0.0497308
[300]	training's l1: 0.0490299	valid_1's l1: 0.0495009
[350]	training's l1: 0.0487859	valid_1's l1: 0.0493208
[400]	training's l1: 0.0485492	valid_1's l1: 0.0491444
[450]	training's l1: 0.0482864	valid_1's l1: 0.0489445
[500]	training's l1: 0.0480441	valid_1's l1: 0.0487677
[550]	training's l1: 0.0478285	valid_1's l1: 0.0486106
[600]	training's l1: 0.0476068	valid_1's l1: 0.0484478
[650]	training's l1: 0.0473987	valid_1's l1: 0.0483021
[700]	training's l1: 0.0472088	valid_1's l1: 0.048168
[750]	training's l1: 0.0470144	valid_1's l1: 0.0480298
[800]	training's l1: 0.0468337	valid_1's l

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')
display_importances(feat_importances)

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
